# PREPROCESSING

Ricette definite in un formato semistrutturato, raggruppamento degli step e ingredienti in un unica stringa, funzioni di utilità

### Funzioni util e import
Import di tutti i requisiti, download dei dataset NLTK e inizializzazione di SpaCy

In [65]:
%%capture

from pathlib import Path
import pandas as pd
from IPython.display import display
from ast import literal_eval
import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import spacy
from spacy import displacy
from spacy.symbols import X, NUM, VERB, NOUN
from nltk.corpus import wordnet as wn

nltk.download('stopwords');
nltk.download('wordnet');
nltk.download('averaged_perceptron_tagger');
nltk.download('maxent_ne_chunker');
nltk.download('words');
nltk.download('conll2000');


!python -m spacy download en_core_web_md

NLP = spacy.load('en_core_web_md');


def string_recipe(i):
    return dataset.iloc[i]['title'] + "\n\n" + dataset.iloc[i]['ingredients'] + "\n\n" + dataset.iloc[i]['step'] 

### Caricamento del dataset

Caricamento del datset da un formato CSV in un DataFrame Pandas

In [66]:
dataset = pd.read_csv(
    Path("../data/test_dataset.csv").resolve(), 
    index_col=[0], 
    names=["index", "title","ingredients","step"], 
    usecols=[0,1,2,3]
    )

for index in range(len(dataset)):
    dataset.iloc[index]['ingredients'] = ".\n".join(literal_eval(dataset.iloc[index]['ingredients']))
    dataset.iloc[index]['step'] = " ".join(literal_eval(dataset.iloc[index]['step']))

display(dataset.head())
print(string_recipe(1626))


,title,ingredients,step
index,,,
0,No-Bake Nut Cookies,1 c. firmly packed brown sugar.\n1/2 c. evapor...,"In a heavy 2-quart saucepan, mix brown sugar, ..."
1,Jewell Ball'S Chicken,"1 small jar chipped beef, cut up.\n4 boned chi...",Place chipped beef on bottom of baking dish. P...
2,Creamy Corn,2 (16 oz.) pkg. frozen corn.\n1 (8 oz.) pkg. c...,"In a slow cooker, combine all ingredients. Cov..."
3,Chicken Funny,1 large whole chicken.\n2 (10 1/2 oz.) cans ch...,Boil and debone chicken. Put bite size pieces ...
4,Reeses Cups(Candy),1 c. peanut butter.\n3/4 c. graham cracker cru...,Combine first four ingredients and press in 13...


Worm Meat(Shredded Barbecue Beef)  

3 lb. or larger chuck roast.
1 large onion, chopped.
1 stalk celery, chopped.
1 large green pepper, chopped.
1 1/2 qt. water.
2 clove garlic.
1 tsp. salt.
1 (16 to 20 oz.) bottle ketchup (plain or tangy).
2 Tbsp. brown sugar.
2 Tbsp. vinegar.
1 tsp. allspice.
1 tsp. dry mustard.
1/2 tsp. chili powder.
2 to 3 drops Tabasco sauce.
1 bay leaf.
1/4 tsp. garlic powder.
1/4 tsp. paprika

Place roast and vegetables in a large, heavy pot. Cover with water. Cook, covered, for 4 hours. Remove meat, cool and shred with a fork. Return meat to 1 1/2 cups of the broth. Add remaining ingredients. Slowly boil, uncovered, for 1 hour. Additional broth may be added, if needed. When done, mixture should be of spreading consistency. Serve on either noodles or rolls. Serves 10 to 12.


### Estrazione delle abbreviazioni

Fase iniziale di ritrovamento del set di abbreviazioni

In [67]:
abbrv_dataset = pd.read_csv(
    Path("../data/test_dataset.csv").resolve(), 
    index_col=[0], 
    names=["index", "title","ingredients","step"], 
    usecols=[0,1,2,3]
    )

abbrv = set()
for index in range(len(dataset)):
    abbrv_dataset.iloc[index]['ingredients'] = " ".join(literal_eval(abbrv_dataset.iloc[index]['ingredients']))
    for element in re.findall(r" [A-Za-z]*\.", abbrv_dataset.iloc[index]['ingredients']):
        abbrv.add(element)
    
print(abbrv)

{' T.', ' Mrs.', ' TBSP.', ' tsp.', ' t.', ' sq.', ' lg.', ' Tbsp.', ' No.', ' mix.', ' tbsp.', ' large.', ' qt.', ' qts.', ' c.', ' pkgs.', ' sm.', ' A.', ' doz.', ' C.', ' lb.', ' etc.', ' gal.', ' lbs.', ' Pkg.', ' pt.', ' m.', ' oz.', ' pkg.'}


Rimozione delle abbreviazioni.

In [68]:
def expand_abbreviations(ingredients_string):
    __ABBREVIATIONS__ = {
        ' pkg.'  :   ' package',
        ' tsb.'  :   ' tablespoon',
        ' no.'   :   ' number',
        ' pt.'   :   ' pint',
        ' no.'   :   ' number',
        ' gal.'  :   ' gallon',
        ' tbsp.' :   ' tablespoon',
        ' sq.'   :   ' square',
        ' oz.'   :   ' ounce',
        ' lb.'   :   ' pound',
        ' qt.'   :   ' quart',
        ' qts.'  :   ' quart',
        ' c.'    :   ' cup',
        ' tsp.'  :   ' teaspoon',
        ' doz.'  :   ' dozen',
        ' c.'    :   ' cup',
        ' pkgs.' :   ' package',  
        ' gal.'  :   ' gallon',
        ' t.'    :   ' teaspoon',
        ' g.'    :   ' gram',
        ' lbs.'  :   ' pound',
    }
    for item, value in __ABBREVIATIONS__.items():
        ingredients_string = ingredients_string.lower().replace(item, value)
    return ingredients_string


for index in range(len(dataset)):
    dataset.iloc[index]['ingredients'] = expand_abbreviations(dataset.iloc[index]['ingredients'])

### Sentence splitting
Il contenuto delle colonne 'ingredients' e 'step' verrà suddiviso in frasi. In precedenza i periodi contenuti nelle singole celle sono stati formattati in modo tale da renderli riconoscibili e facilmente suddivisibili in frasi ben separate.

In [69]:
for index in range(len(dataset)):
     dataset.iloc[index]['ingredients'] = (sent_tokenize(dataset.iloc[index]['ingredients']))
print(dataset.iloc[1626]['ingredients'])

['3 pound or larger chuck roast.', '1 large onion, chopped.', '1 stalk celery, chopped.', '1 large green pepper, chopped.', '1 1/2 quart water.', '2 clove garlic.', '1 teaspoon salt.', '1 (16 to 20 ounce) bottle ketchup (plain or tangy).', '2 tablespoon brown sugar.', '2 tablespoon vinegar.', '1 teaspoon allspice.', '1 teaspoon dry mustard.', '1/2 teaspoon chili powder.', '2 to 3 drops tabasco sauce.', '1 bay leaf.', '1/4 teaspoon garlic powder.', '1/4 teaspoon paprika']


In [70]:
for index in range(len(dataset)):
     dataset.iloc[index]['step'] = (sent_tokenize(dataset.iloc[index]['step']))
print(dataset.iloc[1626]['step'])

['Place roast and vegetables in a large, heavy pot.', 'Cover with water.', 'Cook, covered, for 4 hours.', 'Remove meat, cool and shred with a fork.', 'Return meat to 1 1/2 cups of the broth.', 'Add remaining ingredients.', 'Slowly boil, uncovered, for 1 hour.', 'Additional broth may be added, if needed.', 'When done, mixture should be of spreading consistency.', 'Serve on either noodles or rolls.', 'Serves 10 to 12.']


### Rimozione quantità doppie
Il dataset contiene multiple scelte per le quantità, per facilitare il processo di ritrovamento delle quantità e delle unità di misura eventuali quantità opzionali vengono rimosse.

In [71]:
dr_reg = r'\d*\s*\(.*\)\s'

for index in range(len(dataset)):
    for value in range(len(dataset.iloc[index]['ingredients'])):
        elements = re.findall(dr_reg, dataset.iloc[index]['ingredients'][value])
        for e in elements:
            new_string = dataset.iloc[index]['ingredients'][value].replace(e,e[e.find("(")+1: e.find(")")].strip() + " ")
            dataset.iloc[index]['ingredients'][value] = new_string

In [72]:
print(str(dataset.iloc[1626]['ingredients']))

['3 pound or larger chuck roast.', '1 large onion, chopped.', '1 stalk celery, chopped.', '1 large green pepper, chopped.', '1 1/2 quart water.', '2 clove garlic.', '1 teaspoon salt.', '16 to 20 ounce bottle ketchup (plain or tangy).', '2 tablespoon brown sugar.', '2 tablespoon vinegar.', '1 teaspoon allspice.', '1 teaspoon dry mustard.', '1/2 teaspoon chili powder.', '2 to 3 drops tabasco sauce.', '1 bay leaf.', '1/4 teaspoon garlic powder.', '1/4 teaspoon paprika']


## PREPROCESSING OPZIONALE
Stop word removal, stemming e lemming non hanno dato i risultati sperati nella fase di analisi, sono state rimosse dalla pipeline finale ma vengono definite qui a fine di documentazione dello sviluppo

### Stop words removal
Nella colonna 'step' troviamo una serie di passaggi da compiere per creare la ricetta. Questi passaggi sono scritti in linguaggio naturale e possono essere semplificati rimuovendo delle parole dette stop words.

In [73]:
# OPTINAL -> Risultati in swr_dataset"

stop_words = set(stopwords.words('english'))
swr_dataset = dataset.copy(deep=True)

tk = lambda x,st: ' '.join([w for w in x if w not in st])
for index in range(len(dataset)):
    swr_dataset.iloc[index]['step'] = [tk(word_tokenize(sent), stop_words) for sent in dataset.iloc[index]['step']]

### Stemming e Lemming
Questi due processi potrebbero portare valore all'analisi del dominio. Il codice per entrambi viene proposto qui.

In [74]:
# OPTINAL -> Risultati in lem_dataset"

lemmatizer = WordNetLemmatizer()
lem_dataset = swr_dataset.copy(deep=True)

for index in range(len(swr_dataset)):
    lem_dataset.iloc[index]['step'] = stem_sent = [' '.join([lemmatizer.lemmatize(w) for w in word_tokenize(sent)]) for sent in swr_dataset.iloc[index]['step']]

In [75]:
# OPTINAL -> Risultati in stm_dataset"

stemmer = PorterStemmer()
stm_dataset = lem_dataset.copy(deep=True)

for index in range(len(lem_dataset)):
    stm_dataset.iloc[index]['step'] = [' '.join([stemmer.stem(w) for w in word_tokenize(sent)]) for sent in lem_dataset.iloc[index]['step']]

Stampa dei risultati delle operazioni di stop word removal, stemming e lemming

In [76]:
print("Frasi originali\n")
print(dataset.iloc[10]['step'])
print("\nStop word removal\n")
print(swr_dataset.iloc[10]['step'])
print("\nStop word e lemming\n")
print(lem_dataset.iloc[10]['step'])
print("\nStop word lemming e stemming\n")
print(stm_dataset.iloc[10]['step'])

Frasi originali

['Drain cherries, measuring syrup.', 'Cut cherries in half.', 'Add ginger ale and enough water to syrup to make 1 1/2 cups.', 'Dissolve gelatin in boiling water.', 'Add measured liquid and almond extract.', 'Chill until very thick.', 'Fold in marshmallows and the cherries.', 'Spoon into 6-cup mold.', 'Chill until firm, at least 4 hours or overnight.', 'Unmold.', 'Makes about 5 1/3 cups.']

Stop word removal

['Drain cherries , measuring syrup .', 'Cut cherries half .', 'Add ginger ale enough water syrup make 1 1/2 cups .', 'Dissolve gelatin boiling water .', 'Add measured liquid almond extract .', 'Chill thick .', 'Fold marshmallows cherries .', 'Spoon 6-cup mold .', 'Chill firm , least 4 hours overnight .', 'Unmold .', 'Makes 5 1/3 cups .']

Stop word e lemming

['Drain cherry , measuring syrup .', 'Cut cherry half .', 'Add ginger ale enough water syrup make 1 1/2 cup .', 'Dissolve gelatin boiling water .', 'Add measured liquid almond extract .', 'Chill thick .', 'Fol

# ANALISI
Riconoscimento di ingredienti, step, azioni e relativi attributi

### Entityt Extraction, Relation Extraction e POS tagging con SpaCy
Analisi del dataset tramie il tool SpaCy, funzioni di utilità per la stampa dei risultati di SpaCy

In [77]:
def __pprintsp__(dataset, index, column, value, extend=False):
    displacy.render(dataset.iloc[index][column][value], style='dep')
    print(dataset.iloc[index][column][value])
    displacy.render(dataset.iloc[index][column][value], style='ent')
    if extend:
        for token in dataset.iloc[index][column][value]:
            print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)

def pprint_spacyd_all(dataset, index, column, extend=False):
    for v in range(len(dataset.iloc[index][column])):
        __pprintsp__(dataset, index, column, v, extend)
            
def pprint_spacyd(dataset, index, column, value, extend=False):
    __pprintsp__(dataset, index, column, value, extend)

### Definizione dati strutturati
Classi per step, ricetta e ingredienti

In [78]:
class Recipe:
    def __init__(self, title, index, ingredients_str, steps_str):
        self.title = title
        self.ingredients = []
        self.ing_str = ingredients_str
        self.stp_str = steps_str
        self.steps = []
        self.idx = index
    
    def add_ing(self, ing):
        self.ingredients.append(ing)
        
    def __str__(self):
        _ing_s = '\n'.join([ing.short_srt() for ing in self.ingredients])
        _stp_s = '\n'.join(str(step) for step in self.steps)
        _header_ing = f"|{'NOME':<40s}|{'PROPRIETÀ':<20s}|{'QUANTITÀ':<11s}|{'TIPO':<21s}|"
        _header_ing_l = f"+{'-'*40}+{'-'*20}+{'-'*11}+{'-'*21}+"
        _header_step = f"|{'N°':<2s}|{'AZIONE':<15s}|{'STRUMENTI':<20s}|{'INGREDIENTI':<55s}|"
        _header_step_l = f"+{'-'*2}+{'-'*15}+{'-'*20}+{'-'*55}|"
        return f"TITOLO: {self.title}\n\n{_header_ing}\n{_header_ing_l}\n{_ing_s}\n{_header_ing_l}\n\n{_header_step}\n{_header_step_l}\n{_stp_s}\n{_header_step_l}"
    
    def print_unstructured(self):
        return f"{self.title}\n\n{self.ing_str}\n\n{self.stp_str}"
    
    
class Step:
    def __init__(self, step_str, step_no, index, action, ins, ing):
        self.action = action
        self.ins = ins
        self.ing = ing
        self.step_str = step_str
        self.step_no = step_no
        self.idx = index
        
    def __str__(self):
        return f"|{self.step_no:<2d}|{self.action:<15s}|{','.join(self.ins):20s}|{','.join(self.ing):55s}|"

    
    
class Ingredient:
    def __init__(self, full_text):
        self.name=""
        self.ing_cat_id = None
        self.size=""
        self.quantity= 0.0
        self.size=""
        self.adj=[]
        self.original=full_text
    def __str__(self):
        return f"Name: {self.name}\nAjdectives: {self.adj}\nQuantity: {self.quantity:4f} {self.size}\nOriginal: {self.original}"

    def short_srt(self):
        return f"|{self.name:<40s}|{','.join(self.adj):<20s}|{self.quantity:^11.2f}|{self.size:<21s}|"
    
    def set_id(self, identifier):
        self.ing_cat_id = identifier

## ANALISI INGREDIENTI
Ritrovamento nel testo di:
- Nome dell'ingrediente (es. beef)
- Elenco delle sue proprietà (es. minced)
- Quantità (es. 1.5)
- Misura (es. ounce)

### Funzioni di utilità
Funzioni che permettono di capire se una parola è un cibo tramtite WordNet, pulizia delle stringhe e parsing delle quantità

In [79]:

def is_food(word):
    syns = wn.synsets(str(word), pos = wn.NOUN)
    for syn in syns:
        if 'food' in syn.lexname():
            return True
    return False

def cls_w(word):
    BAD_CH = [',', '.', '(', ')', '*', '+', '-', '\\', '/', '!', '?', '°']
    for ch in BAD_CH:
        word = word.replace(ch, '')
    return word.strip().lower()

def parse_quantity(qt):
    if qt == "":
        return 1
    
    partial_sum = 0.0
    backtrack_multiplier = 1.0
    
    for elem in qt.split(" "):
        try:
        
            if ('/' in elem) and (partial_sum == 0.0):
                parsed = elem.split("/")
                backtrack_multiplier = float(parsed[0])/float(parsed[1])

            elif ('/' in elem) and (partial_sum != 0.0):
                parsed = elem.split("/")
                partial_sum += (float(parsed[0])/float(parsed[1]))

            else:
                partial_sum += float(elem)
        except ValueError:
            pass
            
    if(partial_sum != 0.0):
        partial_sum *= backtrack_multiplier
    else:
        partial_sum = backtrack_multiplier
        
    return partial_sum      

#Root generation

__SIZES__ = [
       'package',
       'tablespoon',
       'number',
       'pint',
       'number',
       'gallon',
       'tablespoon',
       'square',
       'ounce',
       'pound',
       'quart',
       'cup',
       'teaspoon',
       'can',
       'gallon',
       'pound'
]

### Creazione elenco ricette e ingredienti

In [80]:
all_recipes = []
    
for index in range(len(dataset)):
    
    value = round(((index+1)/len(dataset))*20)
    print(f"\rProgress: [{'*'*value}{' '*(20-value)}]", end='')
    
    recipe = Recipe(
        cls_w(dataset.iloc[index]['title']), 
        index, 
        " ".join(dataset.iloc[index]['ingredients']),
        " ".join(dataset.iloc[index]['step'])       
    )
    
    for element in dataset.iloc[index]['ingredients']:

        ingredient = NLP(element)

        IGN = Ingredient(ingredient.text)
        try:
            ROOT_NODE = [token for token in ingredient if token.dep_ == 'ROOT'][0]
        except IndexError:
            print(index)
            print(recipe.title, recipe.ing_str, recipe.step_str)
        

        if (ROOT_NODE.pos == VERB):
                IGN.adj.append(cls_w(ROOT_NODE.text))
        else:
                IGN.name = cls_w(ROOT_NODE.text)
                
        stack = [element for element in ROOT_NODE.children]

        unparsed_quantity = ""
        
        while len(stack)!=0:
            CURRENT_NODE = stack.pop()

            if (CURRENT_NODE.pos == X) or (CURRENT_NODE.pos == NUM):
                    unparsed_quantity = CURRENT_NODE.text + " " + unparsed_quantity
            else:
                if CURRENT_NODE.text.lower() in  __SIZES__:
                    IGN.size += " " + cls_w(CURRENT_NODE.text)

                if ((CURRENT_NODE.dep_ == 'compound') or (CURRENT_NODE.dep_ == 'dobj') or (CURRENT_NODE.dep_ == 'pobj')) and is_food(CURRENT_NODE.text) and (CURRENT_NODE.text.lower() not in  __SIZES__):
                    IGN.name = CURRENT_NODE.text + " " + cls_w(IGN.name)

                if (CURRENT_NODE.dep_ == 'amod') or (CURRENT_NODE.dep_ == 'appos'):
                    if is_food(CURRENT_NODE.text):
                        IGN.name = CURRENT_NODE.text + " " + cls_w(IGN.name)
                    else:
                        IGN.adj.append(cls_w(CURRENT_NODE.text))         

            stack += [element for element in CURRENT_NODE.children]
        
        if IGN.name.strip() != "":
            IGN.quantity = parse_quantity(cls_w(unparsed_quantity))
            IGN.name = cls_w(IGN.name)
            recipe.add_ing(IGN)
    all_recipes.append(recipe)
        

Progress: [********************]

In [81]:
print(all_recipes[1626])

TITOLO: worm meatshredded barbecue beef

|NOME                                    |PROPRIETÀ           |QUANTITÀ   |TIPO                 |
+----------------------------------------+--------------------+-----------+---------------------+
|chuck pound                             |larger              |   3.00    |                     |
|onion                                   |chopped,large       |   1.00    |                     |
|celery                                  |chopped             |   1.00    |                     |
|green                                   |chopped,large       |   1.00    |                     |
|water                                   |                    |   13.00   | quart               |
|clove garlic                            |                    |   2.00    |                     |
|salt                                    |                    |   1.00    | teaspoon            |
|ketchup                                 |plain               |   36.00   | o

## ANALISI STEP
Ritrovamento nel testo delle seguenti informazioni:
- Azione pricipale dello step
- Elenco strumenti utilizzati
- Elenco ingredienti della ricetta coinvolti nello specifico step

### Funzioni di utilità, 
Creazione elenco di tutti gli ingredienti ritrovati nelle ricette, utilizzo di WordNet per ritrovare verbi nascosti (verbi taggati come altri elementi del testo), funzioni per il ritrovamento degli ingredienti della ricetta nello step.

In [82]:
ingredients_all_words = set()

for element in all_recipes:
    for ingredient in element.ingredients:
        for e in ingredient.name.split(" "):
            ingredients_all_words.add(cls_w(e))

In [83]:
def hidden_verb(word):
    ss = wn.synsets(word)
    for s in ss:
        if s.pos() == 'v':
            return True
    return False

def combine_action(action_token):
    action = cls_w(action_token.text)
    for child in action_token.children:
        if child.dep_ == 'prt' or child.dep_ == 'compound':
            action += " " + cls_w(child.text)
    return action




def find_instrument(token):
    stack = [element for element in token.children]
    instrument = ""
    found_main = False
    main_node = None

    while stack:
        s_element = stack.pop()
        if s_element.pos == NOUN and (cls_w(s_element.text) not in ingredients_all_words) and (not found_main) and (s_element.text not in forbidden_instruments):
            if "ed" != s_element.text[-2:] and "ing" != s_element.text[-3:]:
                instrument += cls_w(s_element.text)
                found_main = True
                main_node = s_element
        elif s_element.dep_ == 'compound' and main_node == s_element.head and (cls_w(s_element.text) not in ingredients_all_words):
            instrument = cls_w(s_element.text) + " " + instrument
            main_node = s_element
            
        stack += [element for element in s_element.children]
    return instrument


forbidden_actions = ['done', 'will', 'be']
forbidden_instruments = ['top']
PREPS = ['with', 'in', 'on', 'of', 'using', 'into', 'use']

def get_ingredients(recipe):
    return {ing.name for ing in recipe.ingredients}

def find_ingredients(ings, step):
    
    ing = set()
    for word in step.replace(',','').replace('.','').replace('(','').replace(')','').replace('*','').split(" "):
        word = cls_w(word).replace('+', '').replace('\\', '').replace('!', '')
        for el in ings:
            if re.findall(f"(\\s{word}\\s)|({word}\\s)|(\\s{word})", el) or el == word:
                ing.add(el)                
    return ing


### Aggiornamento dell'elenco ricette con gli step strutturati

In [84]:
step_index = 1
for index in range(len(dataset)):
    step_counter = 1
    
    value = round(((index+1)/len(dataset))*20)
    print(f"\rProgress: [{'*'*value}{' '*(20-value)}]", end='')
    
    ing_s = get_ingredients(all_recipes[index])
    for value in range(len(dataset.iloc[index]['step'])):
        for element in dataset.iloc[index]['step'][value].split(";"):
               
            step = NLP(cls_w(element))
            
            FINAL_ACTION = ""
            _hidden_verbs = []
            _found_action = False
            found_instruments = []
            found_ingredients = []
            
            for TOKEN in step:
                if ("ed" != TOKEN.text[-2:]) and TOKEN.text not in forbidden_actions and ("ing" != TOKEN.text[-3:]):
                    if not _found_action:
                        if (TOKEN.pos == VERB):
                            FINAL_ACTION = combine_action(TOKEN)
                            _found_action = True
                        elif hidden_verb(TOKEN.text):
                            _hidden_verbs.append(combine_action(TOKEN))
                            
                if TOKEN.text in PREPS:
                    instrument = find_instrument(TOKEN)
                    if instrument != "":
                        found_instruments.append(cls_w(instrument))
                        
                      
            if FINAL_ACTION == "" and _hidden_verbs:
                FINAL_ACTION = _hidden_verbs[0]
                
            if FINAL_ACTION != "":
                found_step = Step(element, step_counter, step_index, cls_w(FINAL_ACTION), found_instruments, find_ingredients(ing_s, step.text))
                all_recipes[index].steps.append(found_step)
                step_counter += 1
                step_index += 1

            

Progress: [********************]

In [85]:
print(all_recipes[1626])

TITOLO: worm meatshredded barbecue beef

|NOME                                    |PROPRIETÀ           |QUANTITÀ   |TIPO                 |
+----------------------------------------+--------------------+-----------+---------------------+
|chuck pound                             |larger              |   3.00    |                     |
|onion                                   |chopped,large       |   1.00    |                     |
|celery                                  |chopped             |   1.00    |                     |
|green                                   |chopped,large       |   1.00    |                     |
|water                                   |                    |   13.00   | quart               |
|clove garlic                            |                    |   2.00    |                     |
|salt                                    |                    |   1.00    | teaspoon            |
|ketchup                                 |plain               |   36.00   | o

### Dump 
Tutte le ricette vengono stampate in formato testuale secondo suddivisione in ingredienti, strumenti e step

In [86]:
with open('../dump.txt', 'w') as f:
    for i, index in enumerate(all_recipes):
        f.write(index.print_unstructured() + '\n')
        f.write(f"[{i}]" + str(index) + '\n')

# GENERAZIONE DATABASE

Crezione indici per ricette e ingredienti

In [87]:
macro_ingredients = {}
ingredients_idx = {}
macro_actions = {}
actions_idx = {}
macro_instruments = {}
instruments_idx = {}
actions_idx = {}

for element in all_recipes:
    for ingredient in element.ingredients:
        if ingredient.name in macro_ingredients:
            macro_ingredients[ingredient.name] += 1
        else:
            macro_ingredients[ingredient.name] = 1
            
for index, key in enumerate(macro_ingredients):
    ingredients_idx[key] = index

for element in all_recipes:
    for step in element.steps:
        if step.action in macro_actions:
            macro_actions[step.action] += 1
        else:
            macro_actions[step.action] = 1
            
        for ins in step.ins:
            if ins in macro_instruments:
                macro_instruments[ins] += 1
            else:
                macro_instruments[ins] = 1

for index, key in enumerate(macro_actions):
    actions_idx[key] = index

for index, key in enumerate(macro_instruments):
    instruments_idx[key] = index

Creazione DDL ingredienti

In [88]:
ING_DB_CREATE = """
DROP TABLE IF EXISTS ingredients;
CREATE TABLE ingredients(
id SERIAL PRIMARY KEY,
name VARCHAR NOT NULL
);
"""

print(ING_DB_CREATE)


with open(Path("../database/create_ingredients.sql").resolve(), "w") as ddl_file:
    for ing in ingredients_idx:
        name = ing.replace("'", '')
        ING_DB_CREATE = ING_DB_CREATE + f"INSERT INTO ingredients VALUES ({ingredients_idx[ing]},\'{name}\');\n"
    ddl_file.write(ING_DB_CREATE)


DROP TABLE IF EXISTS ingredients;
CREATE TABLE ingredients(
id SERIAL PRIMARY KEY,
name VARCHAR NOT NULL
);



Creazione DDL strumenti

In [89]:
ING_DB_CREATE = """
DROP TABLE IF EXISTS instruments;
CREATE TABLE instruments(
id SERIAL PRIMARY KEY,
name VARCHAR NOT NULL
);
"""

print(ING_DB_CREATE)


with open(Path("../database/create_instruments.sql").resolve(), "w") as ddl_file:
    for ins in instruments_idx:
        ING_DB_CREATE = ING_DB_CREATE + f"INSERT INTO instruments VALUES ({instruments_idx[ins]},\'{ins}\');\n"
    ddl_file.write(ING_DB_CREATE)


DROP TABLE IF EXISTS instruments;
CREATE TABLE instruments(
id SERIAL PRIMARY KEY,
name VARCHAR NOT NULL
);



Creazione DDL azioni

In [90]:
ING_DB_CREATE = """
DROP TABLE IF EXISTS actions;
CREATE TABLE actions(
id SERIAL PRIMARY KEY,
name VARCHAR NOT NULL
);
"""

print(ING_DB_CREATE)


with open(Path("../database/create_actions.sql").resolve(), "w") as ddl_file:
    for act in actions_idx:
        ING_DB_CREATE = ING_DB_CREATE + f"INSERT INTO actions VALUES ({actions_idx[act]},\'{act}\');\n"
    ddl_file.write(ING_DB_CREATE)


DROP TABLE IF EXISTS actions;
CREATE TABLE actions(
id SERIAL PRIMARY KEY,
name VARCHAR NOT NULL
);



Creazione DDL ricette

In [91]:
REC_DB_CREATE = """
DROP TABLE IF EXISTS recipes;
CREATE TABLE recipes(
id SERIAL PRIMARY KEY,
titolo VARCHAR NOT NULL,
preparazione_nstr VARCHAR,
ingredienti_nstr VARCHAR
);
"""

print(REC_DB_CREATE)



with open(Path("../database/create_recipes.sql").resolve(), "w") as ddl_file:
    for rec in all_recipes:
        
        corrected_ing = rec.ing_str.replace("\'","\'\'")
        corrected_stp = rec.stp_str.replace("\'","\'\'")
        corrected_tit = rec.title.replace("\'","\'\'")
        
        REC_DB_CREATE = REC_DB_CREATE + f"INSERT INTO recipes (id, titolo, ingredienti_nstr, preparazione_nstr) VALUES ({rec.idx},\'{corrected_tit}\',\'{corrected_ing}\',\'{corrected_stp}\');\n"
    ddl_file.write(REC_DB_CREATE)


DROP TABLE IF EXISTS recipes;
CREATE TABLE recipes(
id SERIAL PRIMARY KEY,
titolo VARCHAR NOT NULL,
preparazione_nstr VARCHAR,
ingredienti_nstr VARCHAR
);



Creazione DDL step

In [92]:
CON_DB_CREATE = """
DROP TABLE IF EXISTS steps;
CREATE TABLE steps(
id SERIAL PRIMARY KEY,
descrizione VARCHAR,
numero_step INT,
actionID INT NOT NULL,
FOREIGN KEY (actionID) REFERENCES actions(id)
);
"""

print(CON_DB_CREATE)

with open(Path("../database/create_steps.sql").resolve(), "w") as ddl_file:
    for r in all_recipes:
        for s in r.steps:
            desc = s.step_str.replace("'", '')
            data = f"(\'{desc}\',{s.step_no}, {actions_idx[s.action]})"
            CON_DB_CREATE = CON_DB_CREATE + f"INSERT INTO steps (descrizione, numero_step, actionID) VALUES {data};\n"
    ddl_file.write(CON_DB_CREATE)   


DROP TABLE IF EXISTS steps;
CREATE TABLE steps(
id SERIAL PRIMARY KEY,
descrizione VARCHAR,
numero_step INT,
actionID INT NOT NULL,
FOREIGN KEY (actionID) REFERENCES actions(id)
);



Crezione DDL tabella relazione Ricette <--> Ingredienti

In [93]:
CON_DB_CREATE = """
DROP TABLE IF EXISTS contains;
CREATE TABLE contains(
id SERIAL PRIMARY KEY,
misura VARCHAR,
quantita FLOAT,
proprieta VARCHAR,
recipeID INT NOT NULL,
ingredientID INT NOT NULL,
FOREIGN KEY (recipeID) REFERENCES recipes(id),
FOREIGN KEY (ingredientID) REFERENCES ingredients(id)
);
"""

print(CON_DB_CREATE)


with open(Path("../database/create_contains.sql").resolve(), "w") as ddl_file:
    for r in all_recipes:
        for i in r.ingredients:
            if i.name.strip() != "" and i.size.strip() != "":
                adj = ', '.join(i.adj)
                data = f"(\'{i.size.strip()}\',{i.quantity},\'{adj}\',{r.idx}, {ingredients_idx[i.name]})"
                CON_DB_CREATE = CON_DB_CREATE + f"INSERT INTO contains (misura, quantita, proprieta, recipeID, ingredientID) VALUES {data};\n"
    ddl_file.write(CON_DB_CREATE)      


DROP TABLE IF EXISTS contains;
CREATE TABLE contains(
id SERIAL PRIMARY KEY,
misura VARCHAR,
quantita FLOAT,
proprieta VARCHAR,
recipeID INT NOT NULL,
ingredientID INT NOT NULL,
FOREIGN KEY (recipeID) REFERENCES recipes(id),
FOREIGN KEY (ingredientID) REFERENCES ingredients(id)
);



Crezione DDL tabella relazione Step <--> Strumenti

In [94]:
CON_DB_CREATE = """
DROP TABLE IF EXISTS utilize;
CREATE TABLE utilize(
id SERIAL PRIMARY KEY,
instrumentID INT NOT NULL,
stepID INT NOT NULL,
FOREIGN KEY (instrumentID) REFERENCES instruments(id),
FOREIGN KEY (stepID) REFERENCES steps(id)
);
"""

print(CON_DB_CREATE)


with open(Path("../database/create_utilize.sql").resolve(), "w") as ddl_file:
    for r in all_recipes:
        for s in r.steps:
            for i in s.ins:
                data = f"({instruments_idx[i]}, {s.idx})"
                CON_DB_CREATE = CON_DB_CREATE + f"INSERT INTO utilize (instrumentID, stepID) VALUES {data};\n"
    ddl_file.write(CON_DB_CREATE)  


DROP TABLE IF EXISTS utilize;
CREATE TABLE utilize(
id SERIAL PRIMARY KEY,
instrumentID INT NOT NULL,
stepID INT NOT NULL,
FOREIGN KEY (instrumentID) REFERENCES instruments(id),
FOREIGN KEY (stepID) REFERENCES steps(id)
);



Crezione DDL tabella relazione Step <--> Ricette

In [95]:
CON_DB_CREATE = """
DROP TABLE IF EXISTS composed;
CREATE TABLE composed(
id SERIAL PRIMARY KEY,
recipeID INT NOT NULL,
stepID INT NOT NULL,
FOREIGN KEY (recipeID) REFERENCES recipes(id),
FOREIGN KEY (stepID) REFERENCES steps(id)
);
"""

print(CON_DB_CREATE)


with open(Path("../database/create_composed.sql").resolve(), "w") as ddl_file:
    for r in all_recipes:
        for s in r.steps:
            data = f"({r.idx}, {s.idx})"
            CON_DB_CREATE = CON_DB_CREATE + f"INSERT INTO composed (recipeID, stepID) VALUES {data};\n"
    ddl_file.write(CON_DB_CREATE)  


DROP TABLE IF EXISTS composed;
CREATE TABLE composed(
id SERIAL PRIMARY KEY,
recipeID INT NOT NULL,
stepID INT NOT NULL,
FOREIGN KEY (recipeID) REFERENCES recipes(id),
FOREIGN KEY (stepID) REFERENCES steps(id)
);



Crezione DDL tabella relazione Step <--> Ingredienti

In [96]:
CON_DB_CREATE = """
DROP TABLE IF EXISTS involves;
CREATE TABLE involves(
id SERIAL PRIMARY KEY,
ingredientID INT NOT NULL,
stepID INT NOT NULL,
FOREIGN KEY (ingredientID) REFERENCES ingredients(id),
FOREIGN KEY (stepID) REFERENCES steps(id)
);
"""

print(CON_DB_CREATE)


with open(Path("../database/create_involves.sql").resolve(), "w") as ddl_file:
    for r in all_recipes:
        for s in r.steps:
            for i in s.ing:
                data = f"({ingredients_idx[i]}, {s.step_no})"
                CON_DB_CREATE = CON_DB_CREATE + f"INSERT INTO involves (ingredientID, stepID) VALUES {data};\n"
    ddl_file.write(CON_DB_CREATE)  


DROP TABLE IF EXISTS involves;
CREATE TABLE involves(
id SERIAL PRIMARY KEY,
ingredientID INT NOT NULL,
stepID INT NOT NULL,
FOREIGN KEY (ingredientID) REFERENCES ingredients(id),
FOREIGN KEY (stepID) REFERENCES steps(id)
);

